In [1]:
pip install --upgrade google-cloud-speech

    100% |████████████████████████████████| 92kB 49kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 62kB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 10kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 15kB/s ta 0:00:014
    100% |████████████████████████████████| 2.0MB 20kB/s ta 0:00:018
    100% |████████████████████████████████| 102kB 15kB/s a 0:00:01
    100% |████████████████████████████████| 81kB 18kB/s ta 0:00:01
  Stored in directory: /Users/rohanjohn/Library/Caches/pip/wheels/9e/3d/a2/1bec8bb7db80ab3216dbc33092bb7ccd0debfb8ba42b5668d5
Successfully built googleapis-common-protos
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade google-cloud-storage

    100% |████████████████████████████████| 71kB 22kB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.getcwd()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="api-key.json"

In [2]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install soundfile

Note: you may need to restart the kernel to use updated packages.


In [41]:
import librosa
import soundfile as sf

In [12]:
filepath = "/Users/rohanjohn/Documents/music/"     #Input audio file path
output_filepath = "/Users/rohanjohn/Documents/Transcripts"#Final transcript path
bucketname = "callcenteraudio" #Name of the bucket created in the step before


from pydub import AudioSegment
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import wave
from google.cloud import storage

In [2]:
import os
import speech_recognition as sr
from tqdm import tqdm

with open("api-key.json") as f:
    GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()

In [3]:

from google.oauth2 import service_account
import os
credential = service_account.Credentials.from_service_account_file("/Users/rohanjohn/Downloads/neat-beaker-249716-9490dd4bfd30.json")
print (credentials)
#client = language.LanguageServiceClient(credentials=credentials)

NameError: name 'credentials' is not defined

In [3]:
def mp3_to_wav(audio_file_name):
    if audio_file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(audio_file_name)
        audio_file_name = audio_file_name.split('.')[0] + '.wav'
        sound.export(audio_file_name, format="wav")


In [4]:
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [5]:
def frame_rate_channel(audio_file_name):
   # x,_ = librosa.load(audio_file_name, sr=16000)
   # sf.write('tmp.wav', x, 16000)
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [6]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [7]:
def delete_blob(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [8]:
def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
    mp3_to_wav(file_name)

    # The name of the audio file to transcribe
    
    frame_rate, channels = frame_rate_channel(file_name)
    
    if channels > 1:
        stereo_to_mono(file_name)
    
    bucket_name = bucketname
    source_file_name = filepath + audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='hi-IN')

    # Detects speech in the audio file
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)

    for result in response.results:
        transcript += result.alternatives[0].transcript
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript

    
    

In [9]:

def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+")
    f.write(transcript)
    f.close()

In [10]:
os.listdir(filepath)

['hindicall.wav', '.DS_Store']

In [11]:
import os
print (os.environ['PATH'])

/anaconda3/bin:/anaconda3/bin:/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin


In [99]:
pip install ffmpeg

  Stored in directory: /Users/rohanjohn/Library/Caches/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
Note: you may need to restart the kernel to use updated packages.


In [13]:
if __name__ == "__main__":
    for audio_file_name in os.listdir(filepath):
        transcript = google_transcribe(audio_file_name)
        transcript_filename = audio_file_name.split('.')[0] + '.txt'
        write_transcripts(transcript_filename,transcript)

In [14]:
print(transcript)

सुप्रभात क्या मेरी बात सुश्री प्रभाकर से हो रही है हां बात कर रही हूं मेरा नाम सुरभि है और मैं हाल ही में आपके लिए भेजे गए एक महत्वपूर्ण प्रस्ताव के बारे में रैली गेट की तरफ से फोन कर रही हूं हम प्रतिक्रियाओं की समीक्षा कर रहे हैं और ध्यान दिया है कि हमें आप की प्रतिकृति अभी तक नहीं मिली है जैसा कि इस ऑफर की समाप्ति की तारीख नजदीक आ रही है इसलिए मैं आपके अपने बीमा कवर को बढ़ाने की शानदार अक्षर के बारे में बताना चाहूंगी जारी रखने से पहले मुझे आपको सूचित करना होगा कि यह कॉल गुणवत्ता आश्वासन के लिए रिकॉर्ड किया जा रहा है यह आपके लिए उचित है हां श्री प्रभाकर जैसे कि आप पहले आप पहले से ही स्वास्थ्य बीमा होने का मूल्य जानती है जैसा कि इस अनिश्चित काल में हमें लगता है कि हमारे पॉलिसी धारकों के लिए अपने परिवार की जरूरतों को पूरा करने हेतु पर्याप्त दुर्घटना बीमा का होना अधिक महत्वपूर्ण है आपको हाल ही में मेरे द्वारा हमने एक पत्र हमसे एक पत्र प्राप्त हुआ होगा तेरे लिए आपको एक विशिष्ट बीमा उत्पाद प्रदान कर रहा है जिसे सिक्योर पथएक्सीडेंट कहा जाता है आकस्मिक मृत्यु कवर के अंतर्गत 10 लाख की धनराशि 

In [16]:
pip install googletrans

  Stored in directory: /Users/rohanjohn/Library/Caches/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans
Note: you may need to restart the kernel to use updated packages.


In [17]:
from googletrans import Translator

In [20]:
 t = Translator()
translated =  t.translate(transcript)

In [21]:
print(translated)

Translated(src=hi, dest=en, text=Good morning, I am talking to Ms. Prabhakar, yes I am talking, my name is Surabhi and I am calling on behalf of rally gate about an important proposal sent to you recently. We are reviewing the responses. And have noticed that we have not received your replica as of now, as the expiry date of this offer is approaching, so I am brilliant to increase your own insurance cover I would like to inform you about the letter before continuing. This call is being recorded for quality assurance. It is appropriate for you. Yes Mr. Prabhakar as if you already know the value of having health insurance already. As we feel in this indefinite period, our policy holders have enough accident insurance to meet the needs of their family. It is important that you have received a letter from me recently. I am providing you with a specific insurance product called Secure Path Accident under the Accidental Death Cover for a sum of Rs. 10 lakh plus the policy normal amount. One 

In [39]:
pip install google-cloud-language

    100% |████████████████████████████████| 92kB 69kB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
from google.cloud import language

In [42]:
pip install --upgrade requests google-cloud-language

    100% |████████████████████████████████| 61kB 42kB/s ta 0:00:016
Requirement already up-to-date: google-cloud-language in /anaconda3/lib/python3.7/site-packages (1.3.0)
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0
Note: you may need to restart the kernel to use updated packages.


In [24]:
from google.cloud.language import types

In [49]:
pip install --upgrade google-cloud-language

Requirement already up-to-date: google-cloud-language in /anaconda3/lib/python3.7/site-packages (1.3.0)
Note: you may need to restart the kernel to use updated packages.


In [28]:
def language_analysis(text):
    client = language.LanguageServiceClient() 
    document = language.types.Document(content=text, type='PLAIN_TEXT')
    sent_analysis = client.analyze_sentiment(document=document).document_sentiment
    print(dir(sent_analysis)) 
    entities = client.analyze_entities(document).entities
    return sent_analysis, entities


sentiment, entities = language_analysis(translated.text)
print('Text: {}'.format(translated.text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))
#print(sentiment.score, sentiment.magnit)

entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION', 'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

for e in entities:
        print(('name', e.name), ('type', entity_type[e.type]), ('metadata', e.metadata), ('salience', e.salience))

['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_SetListener', '__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', '_extensions_by_name', '_extensions_by_number', 'magnitude', 'score']
Text: Good morning, I am talking to Ms. Prabhakar, yes I am talking, my name is Surabhi and I am calli

IndexError: tuple index out of range

In [22]:
def google_transcribe_dir(audio_file_name):
    
    file_name = filepath + audio_file_name
    mp3_to_wav(file_name)

    # The name of the audio file to transcribe
    
    frame_rate, channels = frame_rate_channel(file_name)
    
    if channels > 1:
        stereo_to_mono(file_name)
    
    bucket_name = bucketname
    source_file_name = filepath + audio_file_name
    destination_blob_name = audio_file_name
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + audio_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='hi-IN',
    enable_speaker_diarization=True,
    diarization_speaker_count=2)
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)
    result = response.results[-1]
    words_info = result.alternatives[0].words
    
    tag=1
    speaker=""

    for word_info in words_info:
        if word_info.speaker_tag==tag:
            speaker=speaker+" "+word_info.word
        else:
            transcript += "speaker {}: {}".format(tag,speaker) + '\n'
            tag=word_info.speaker_tag
            speaker=""+word_info.word

    transcript += "speaker {}: {}".format(tag,speaker)
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript


In [21]:
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types

In [23]:
if __name__ == "__main__":
    for audio_file_name in os.listdir(filepath):
        
        
        transcript1 = google_transcribe_dir(audio_file_name)
        transcript_filename = audio_file_name.split('.')[0] + '_di.txt'
        write_transcripts(transcript_filename,transcript1)

In [24]:
print(transcript1)

speaker 1: 
speaker 2: I think we got disconnected this is re calling you back from satellites. I just kind of hurts like going to silent so I don't know what you're not you might have switched over to another line or if maybe you could hear me I was talking and I couldn't hear you so I was just calling back just to make sure that you know we didn't get disconnected by
speaker 1: accident sir yeah we did okay well I mean you do
speaker 2: qualify for the some great promotion for you looking to get service set up sooner rather
speaker 1: than later no I'm watching surprised I'm checking prices I'm going to see call some other places and see what the other prices I can get
speaker 2: that's fine but we are located pretty much out there so we can actually if you are going to shop around you welcome to do that but if you reserve the prices weather today Ryan that would actually allow me to guarantee the prices for 30 days so when you found out that there's not going to be really any other 